In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os

if os.getcwd().split('/')[-1] == 'data':
    os.chdir('..')

from util import get_openml_dataset

# credit card default
https://www.kaggle.com/uciml/default-of-credit-card-clients-dataset

In [ ]:
!gdown 'https://drive.google.com/uc?id=1yff8lGjjBOUB_-M4FJOolXgnBX_akfiZ'
!unzip UCI_Credit_Card.zip
!rm UCI_Credit_Card.zip
!mkdir data/credit_card
!mv UCI_Credit_Card.csv data/credit_card

In [ ]:
df_raw = pd.read_csv("data/credit_card/UCI_Credit_Card.csv")

df_raw.columns = [col.lower() for col in df_raw.columns]
orig_feature_count = len(df_raw.columns) - 2
orig_feature_count

In [ ]:
categorical_features = ['sex', 'education', 'marriage']
df_enc = pd.get_dummies(df_raw, columns=categorical_features, prefix_sep=':')

In [ ]:
X, y = df_enc.drop(['id', 'default.payment.next.month'], axis=1), df_enc['default.payment.next.month']
df_tgt_last = pd.concat((X, y), axis=1)

In [ ]:
df_tgt_last.to_csv("data/credit_card/credit_card_clean.csv", index=False)

# recidivism
https://github.com/propublica/compas-analysis

In [ ]:
!git -C data clone https://github.com/propublica/compas-analysis

In [ ]:
df_raw = pd.read_csv("data/compas-analysis/compas-scores-two-years.csv")
df_raw['c_jail_time'] = (pd.to_datetime(df_raw['c_jail_out']) - pd.to_datetime(df_raw['c_jail_in'])).dt.days
cols_interest = ['id', 'age', 'c_charge_degree', 'race', 'age_cat', 'sex', 
                 'priors_count', 'days_b_screening_arrest', 'is_recid', 
                 'c_jail_in', 'c_jail_out', 'c_jail_time',
                 'juv_fel_count', 'juv_other_count', 'juv_misd_count']
df = df_raw[cols_interest]

### follow same filtering process as propublica analysis

In [ ]:
df = df[df['is_recid'] != -1]
df = df[df['c_charge_degree'] != 'O']
df = df[df['days_b_screening_arrest'].abs() <= 30]
df = df.drop(['c_jail_in', 'c_jail_out'], axis=1)

In [ ]:
orig_feature_count = len(df.columns) - 1
orig_feature_count

In [ ]:
df_enc = pd.get_dummies(df, prefix_sep=':')
df_enc.columns = df_enc.columns.str.replace(' ', '_')
X, y = df_enc.drop(['id', 'is_recid'], axis=1), df_enc['is_recid']

df_tgt_last = pd.concat((X, y), axis=1)

In [ ]:
df_tgt_last.to_csv('data/compas-analysis/compas_two_year_clean.csv', index=False)

# juvenile 
https://www.icpsr.umich.edu/web/NACJD/studies/3986

In [ ]:
!gdown 'https://drive.google.com/uc?id=1wEFXutadmevTt1PUpjaDv4XH9KkSMdbx'
!unzip ICPSR_03986.zip
!rm ICPSR_03986.zip
!mv ICPSR_03986 data/ICPSR_03986

### create df from raw txt data   

In [ ]:
raw_rows = open('data/ICPSR_03986/DS0001/03986-0001-Data.txt').read().split('\n')

# consolidated info from 03986-0001-Codebook/Questionnaire.pdf and 03986-0001-User_guide.pdf
metadata = pd.read_csv('data/ICPSR_03986/DS0001/feature_info.csv')

In [ ]:
rows = [[] for _ in range(len(raw_rows) - 1)]

for i in range(len(rows)):
    for j, l in zip(metadata['start_ind'], metadata['length']):
        rows[i].append(raw_rows[i][j:j+l])

for i in range(len(rows)):
    rows[i] = list(map(lambda x: x.strip(), rows[i]))

In [ ]:
df = pd.DataFrame(rows)
df.columns = metadata['feature_name'].values
df.shape

### clean missing values

In [ ]:
metadata['missing_vals_set'] = metadata['missing_val'].astype(str) + ' ' + metadata['missing_val_2'].astype(str)
metadata['missing_vals_set'] += ' ' + metadata['missing_val_3'].astype(str)
metadata['missing_vals_set'] = (
    metadata['missing_vals_set'].apply(lambda x: set([v[:-2] for v in x.split(' ') if v != 'nan']))
)

In [ ]:
df = df.loc[:, ~metadata['over_10_percent_missing'].values]
df.shape

In [ ]:
rem_col_missing_val_sets = metadata[~metadata['over_10_percent_missing']]['missing_vals_set']
for i in range(df.shape[1]):
    curr_feat_missing_values = rem_col_missing_val_sets.iloc[i]
    df = df[~df.iloc[:, i].isin(curr_feat_missing_values)]

### separate outcome variables

In [ ]:
outcome_variables = metadata['feature_name'][
    metadata['delinquent_behavior'].astype(bool) & metadata['feature_name'].isin(df.columns)
]
drop_variables = list(outcome_variables) + ['id', 'any_deviance']
X_cat, y = df.drop(drop_variables, axis=1), df['any_deviance']

In [ ]:
orig_feature_count = len(X_cat.columns)
orig_feature_count

### encode categorical features

In [ ]:
categorical_features = metadata['feature_name'][
    metadata['categorical'].astype(bool) & metadata['feature_name'].isin(X_cat.columns)
]

In [ ]:
X = pd.get_dummies(X_cat, columns=categorical_features, prefix_sep=':').astype('float32')
y = y.astype('float32')

In [ ]:
df_clean = pd.concat([X, y], axis=1)
df_clean.to_csv('data/ICPSR_03986/DS0001/data_clean.csv', index=False)

# diabetes readmission
https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008

In [ ]:
!gdown 'https://drive.google.com/uc?id=1rOoEbLjMYsyamKeq4WFwHNdSZXPcRQUH'
!unzip readmission.zip
!rm readmission.zip
!mv readmission data/readmission

In [ ]:
df = pd.read_csv('data/readmission/diabetic_data.csv')
df = df.replace({'?': np.nan})

### target

In [ ]:
df['readmitted'] = df['readmitted'].replace({'NO': 0, '>30': 1, '<30': 1})

### race

In [ ]:
df['race'] = df['race'].replace({np.nan: 'Other'})
df = pd.get_dummies(df, columns=['race'], prefix_sep=':')

### gender

In [ ]:
df = df[df['gender'] != 'Unknown/Invalid']
df = pd.get_dummies(df, columns=['gender'], prefix_sep=':')

### age

In [ ]:
df['age'] = df['age'].replace({"[70-80)":"70+",
                               "[60-70)":"[50-70)",
                               "[50-60)":"[50-70)",
                               "[80-90)":"70+",
                               "[40-50)":"[20-50)",
                               "[30-40)":"[20-50)",
                               "[90-100)":"70+",
                               "[20-30)":"[20-50)"})
df = pd.get_dummies(df, columns=['age'], prefix_sep=':')

### admission type id

In [ ]:
df['admission_type_id'] = df['admission_type_id'].replace({1.0:"Emergency",
                                                           2.0:"Emergency",
                                                           3.0:"Elective",
                                                           4.0:"New Born",
                                                           5.0:np.nan,
                                                           6.0:np.nan,
                                                           7.0:"Trauma Center",
                                                           8.0:np.nan})
df = pd.get_dummies(df, columns=['admission_type_id'], prefix_sep=':')

### discharge disposition ID

In [ ]:
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(
    {1:"Discharged to Home",
     6:"Discharged to Home",
     8:"Discharged to Home",
     13:"Discharged to Home",
     19:"Discharged to Home",
     18:np.nan, 25:np.nan, 26:np.nan,
     2:"Other", 3:"Other", 4:"Other",
     5:"Other", 7:"Other", 9:"Other",
     10:"Other", 11:"Other", 12:"Other",
     14:"Other", 15:"Other", 16:"Other",
     17:"Other", 20:"Other", 21:"Other",
     22:"Other", 23:"Other", 24:"Other",
     27:"Other", 28:"Other", 29:"Other", 30:"Other"}
) 
df = pd.get_dummies(df, columns=['discharge_disposition_id'], prefix_sep=':')

### admission source ID

In [ ]:
df['admission_source_id'] = df['admission_source_id'].replace(
    {1:"Referral", 2:"Referral", 3:"Referral", 4:"Transfer",
     5:"Transfer", 6:"Transfer", 7:"Emergency", 8:"Other",
     9:"Other", 10:"Transfer", 11:"Other", 12:"Other",
     13:"Other", 14:"Other", 15:np.nan, 17:np.nan, 
     18:"Transfer", 19:"Other", 20:np.nan, 21:np.nan,
     22:"Transfer", 23:"Other", 24: "Other", 25:"Transfer",
     26: "Transfer"}
)
df = pd.get_dummies(df, columns=['admission_source_id'], prefix_sep=':')

### medical specialty

In [ ]:
df['medical_specialty'] = df['medical_specialty'].replace(
    {"Orthopedics-Reconstructive": "Orthopedics",
     "Surgeon": "Surgery-General",
     "Surgery-Cardiovascular": "Surgery-Cardiovascular/Thoracic",
     "Surgery-Thoracic": "Surgery-Cardiovascular/Thoracic",
     "Pediatrics-Endocrinology": "Pediatrics",
     "Pediatrics-CriticalCare": "Pediatrics",
     "Pediatrics-Pulmonology": "Pediatrics",
     "Radiologist": "Radiology",
     "Oncology": "Hematology/Oncology",
     "Hematology": "Hematology/Oncology",
     "Gynecology": "ObstetricsandGynecology",
     "Obstetrics": "ObstetricsandGynecology"
     }
)
df['medical_specialty'] = df['medical_specialty'].replace(
    {spec: "Other" for spec in df['medical_specialty'].value_counts().index.values[15:]}
)
df = pd.get_dummies(df, columns=['medical_specialty'], prefix_sep=':')

### diagnosis

In [ ]:
def map_diagnosis(data, cols):
    for col in cols:
        data.loc[(data[col].str.contains("V")) | (data[col].str.contains("E")), col] = -1
        data[col] = data[col].astype(np.float16)

    for col in cols:
        data["temp_diag"] = np.nan
        data.loc[(data[col]>=390) & (data[col]<=459) | (data[col]==785), "temp_diag"] = "Circulatory"
        data.loc[(data[col]>=460) & (data[col]<=519) | (data[col]==786), "temp_diag"] = "Respiratory"
        data.loc[(data[col]>=520) & (data[col]<=579) | (data[col]==787), "temp_diag"] = "Digestive"
        data.loc[(data[col]>=680) & (data[col]<=709) | (data[col]==782), "temp_diag"] = "Skin"
        data.loc[(data[col]>=240) & (data[col]<250) | (data[col]>251) & (data[col]<=279), "temp_diag"] = "Non-diabetes endocrine/metabolic"
        data.loc[(data[col]>=250) & (data[col]<251), "temp_diag"] = "Diabetes"
        data.loc[(data[col]>=800) & (data[col]<=999), "temp_diag"] = "Injury"
        data.loc[(data[col]>=710) & (data[col]<=739), "temp_diag"] = "Musculoskeletal"
        data.loc[(data[col]>=580) & (data[col]<=629) | (data[col] == 788), "temp_diag"] = "Genitourinary"
        data.loc[(data[col]>=140) & (data[col]<=239), "temp_diag"] = "Neoplasms"
        data.loc[(data[col]>=290) & (data[col]<=319), "temp_diag"] = "Mental"
        data.loc[(data[col]>=1) & (data[col]<=139), "temp_diag"] = "Infectious"

        data["temp_diag"] = data["temp_diag"].fillna("Other")
        data[col] = data["temp_diag"]
        data = data.drop("temp_diag", axis=1)

    return data

In [ ]:
df = map_diagnosis(df, ["diag_1","diag_2","diag_3"])
df = pd.get_dummies(df, columns=["diag_1","diag_2","diag_3"], prefix_sep=':')

### medications

In [ ]:
df['diabetesMed'] = df['diabetesMed'].replace({'Yes': 1, 'No': 0})

In [ ]:
df['change'] = df['change'].replace({'Ch': 1, 'No': 0})

In [ ]:
all_meds = df.columns[14:37]
keep_meds = all_meds.values[
    [(df[med].value_counts().shape[0] > 1) and (df[med].value_counts()['Steady'] > 30) for med in all_meds]
]
drop_meds = all_meds.values[~all_meds.isin(keep_meds)]
keep_meds

In [ ]:
df = pd.get_dummies(df, columns=keep_meds, prefix_sep=':')

### test results

In [ ]:
df = pd.get_dummies(df, columns=['A1Cresult', 'max_glu_serum'], prefix_sep=':')

### final

In [ ]:
drop_columns = ['encounter_id', 'patient_nbr', 'weight', 'payer_code'] + drop_meds.tolist()

X, y = df.drop(drop_columns + ['readmitted'], axis=1), df['readmitted']
df_clean = pd.concat([X, y], axis=1)
df_clean.to_csv('data/readmission/readmission_clean.csv', index=False)

orig_feature_count = len(pd.read_csv('data/readmission/diabetic_data.csv').columns) - len(drop_columns) - 1
orig_feature_count

# breast cancer
https://www.openml.org/d/13

In [ ]:
df = get_openml_dataset(13)
df = df.dropna()

categorical_features = ['menopause', 'breast-quad', 'deg-malig']
df_enc = pd.get_dummies(df, columns=categorical_features)

X, y = df_enc.drop('recurrence-events', axis=1), df_enc['recurrence-events']
df_clean = pd.concat([X, y], axis=1)
df_clean.to_csv('data/breast_cancer.csv', index=False)

orig_feature_count = len(df.columns) - 1
orig_feature_count

# german credit
https://www.openml.org/d/31

In [ ]:
df = get_openml_dataset(31)

categorical_features = ['checking_status', 'credit_history', 'purpose', 'savings_status', 'employment', 
                        'personal_status', 'other_parties', 'property_magnitude', 'other_payment_plans',
                        'housing', 'job', 'num_dependents']
df_enc = pd.get_dummies(df, columns=categorical_features)

X, y = df_enc.drop('good', axis=1), df_enc['good']
df_clean = pd.concat([X, y], axis=1)
df_clean.to_csv('data/credit_g.csv', index=False)

orig_feature_count = len(df.columns) - 1
orig_feature_count

# haberman
https://www.openml.org/d/43

In [ ]:
df = get_openml_dataset(43)
df.to_csv('data/haberman.csv', index=False)
orig_feature_count = len(df.columns) - 1
orig_feature_count

# heart
https://www.openml.org/d/1574

In [ ]:
df = get_openml_dataset(1574)
df = df.rename(columns={1.0:'target'})

categorical_features = ['att_13']

df_enc = pd.get_dummies(df, columns=categorical_features)

for col in [f'att_{i}' for i in [2, 6, 7, 9, 11]]:
    df_enc[col] = (df_enc[col] == 1.0).astype(int)

X, y = df_enc.drop('target', axis=1), df_enc['target']
df_clean = pd.concat([X, y], axis=1)
df_clean.to_csv('data/heart.csv', index=False)

orig_feature_count = len(df.columns) - 1
orig_feature_count